In [1]:
import numpy as np
import matplotlib.pyplot as plt

from fastprogress import progress_bar

from runningstatistics import StatsTracker
import jlinops 
import sampi

In [2]:
class CGLSGaussianSampler:
    """Represents a Gaussian sampler for Gaussians of the form
    -log \pi(x) = \sum_i^K || L_i x - s_i ||_2^2 + C.
    
    """
    
    def __init__(self, factors, shifts):
        
        self.factors = factors
        self.shifts = shifts
            
        # Checks
        self.n = self.factors[0].shape[1]
        for factor in self.factors[1:]:
            assert factor.shape[1] == self.n, "incompatible shapes for factors."
        
        # Set None shifts to zeros
        for j in len(self.shifts):
            if self.shifts[j] is None:
                self.shifts[j] = np.zeros(self.factors[j].shape[0])
                
        # Assemble matrix
        self.A = jlinops.StackedOperator(self.factors)
        self.m = self.A.shape[0]
        
        # Assemble deterministic part of rhs
        self.rhs_det = np.hstack(self.shifts)
        
        
    def sample(self, n_samples=100, *args, **kwargs):
        
        # Instantiate tracker
        tracker = StatsTracker((n,))
        
        # Generate samples
        for j in range(n_samples):
            
            # Generate random part of rhs
            rhs_rand = np.random.normal(size=self.m)
            
            # Sum together
            rhs = self.rhs_det + rhs_rand
            
            # Solve the random least-squares problem
            cgls_solve = jlinops.cgls(self.A, rhs, *args, **kwargs)
            sample = cgls_solve["x"]
            tracker.push(sample)
            
        data = {
            "mean": tracker.mean(),
            "stdev": tracker.stdev(),
            "var": tracker.var(),
        }
        
        return data
            